In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import napari
from PIL import Image

from scribbles_creator import *
from cellpose_data_handler import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Define where the images are located and what data to consider

In [ ]:
folder_path = "./cellpose_train_imgs/24-03-08_pred01"

In [ ]:
mode = "all"
bins = [0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1]
all_suff = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
suff = all_suff[:5]

## Create predictions/segmentations on the cellpose dataset (with self-created scribbles)

In [ ]:
# Define the convpaint parameters
layer_list = [0] # layers to use for convpaint (depending on the model chosen)
scalings = [1, 2] # scalings for convpaint (downscaling the image)
model="vgg16" # 'vgg16', 'efficient_netb0', 'single_layer_vgg16', 'single_layer_vgg16_rgb', 'dino_vits16'
random_state = None # seed used for random forest classifier

# Predictions
for img_num in range(0, 10):
    for bin in bins:
        for s in suff:
            print(f"IMG {img_num}: {bin}_{s}")
            pred_cellpose_convpaint(folder_path, img_num, mode=mode, bin=bin, suff=s, layer_list=layer_list, scalings=scalings, model=model, random_state=random_state, save_res=True, show_res=False)
            pred_cellpose_ilastik(folder_path, img_num, mode=mode, bin=bin, suff=s, save_res=True, show_res=False)

Same cell but for testing...

In [4]:
folder_path = "./results/pred03_examples"
# folder_path = "./cellpose_train_imgs"

mode = "all"
bins = [0.1]#, 0.025, 0.05, 0.1, 0.25, 0.5, 1]
all_suff = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
suff = all_suff[:1]

# Define the convpaint parameters
layer_list = [0] # layers to use for convpaint (depending on the model chosen)
scalings = [1, 2] # scalings for convpaint (downscaling the image)
model="single_layer_vgg16" # 'vgg16', 'efficient_netb0', 'single_layer_vgg16', 'single_layer_vgg16_rgb', 'dino_vits16'
random_state = 123 # seed used for random forest classifier

# Predictions
for img_num in (0,189):#range(0, 10):
    for bin in [0.01]: #bins:
        for s in ["a"]:#suff:
            print(f"IMG {img_num}: {bin}_{s}")
            # get_cellpose_img_data(folder_path, img_num, load_img=True, load_gt=False, load_scribbles=False, mode=mode, bin=bin, suff=s, load_pred=False, pred_tag="convpaint")
            pred_cellpose_convpaint(folder_path, img_num, mode=mode, bin=bin, suff=s, layer_list=layer_list, scalings=scalings, model=model, random_state=random_state, save_res=False, show_res=True)
            pred_cellpose_ilastik(folder_path, img_num, mode=mode, bin=bin, suff=s, save_res=False, show_res=True)

IMG 0: 0.01_a
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 383, 512)
IMG 189: 0.01_a
Active channels: R=False, G=True, B=False --> Removed 2 channel(s) --> shape: (1, 209, 267)


In [ ]:

### VGG16 model choices: 'vgg16', 'single_layer_vgg16', 'single_layer_vgg16_rgb' (others: 'efficient_netb0', 'dino_vits16')
# if model_name == 'vgg16':
#     self.model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
#     # self.transform =  models.VGG16_Weights.IMAGENET1K_V1.transforms()
# elif model_name == 'efficient_netb0':
#     self.model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
# elif model_name == 'single_layer_vgg16':
#     self.model = load_single_layer_vgg16(keep_rgb=False)
# elif model_name == 'single_layer_vgg16_rgb':
#     self.model = load_single_layer_vgg16(keep_rgb=True)
# elif model_name == 'dino_vits16':
#     self.model = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')

# TEST ILASTIK

In [ ]:
folder_path = "./cellpose_train_imgs/24-03-08_pred01"
img = np.array(Image.open(f"{folder_path}/000_img.png")) # NOTE: we need to take only one channel for Ilastik to work (so far)
print(len(img.shape))
# img = np.moveaxis(img, -1, 0)
# print(img.shape)
labels = np.array(Image.open(f"{folder_path}/000_scribbles_all_00100_a.png"))
print(len(labels.shape))

In [ ]:
v = napari.Viewer()
v.add_image(img, name="img")
v.add_labels(labels, name="labels")

In [ ]:
pred = pixel_classification_ilastik(img, labels)

In [ ]:
v.add_labels(pred, name="Ilastik")

In [ ]:
v.add_labels(np.array(Image.open(f"{folder_path}/000_convpaintCh1_all_0.1_a.png")), name="Convpaint")